In [1]:
import sys
import pickle
from scipy import signal
from scipy import stats
import numpy as np
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import cohen_kappa_score

import math
from collections import OrderedDict


import matplotlib.pyplot as plt

sys.path.append('D:\Diamond\code')
from csp_james_2 import *

sys.path.append('D:\Diamond\code')
from thesis_funcs_19_03 import *

import torch
import torch.nn as nn
import torch.nn.functional as nnF
import torch.optim as optim
from torch.autograd import Variable
from torch.optim import lr_scheduler

import csv
from random import randint
import random
import datetime

In [2]:
save_root  = 'E:\\Diamond\\bci_iv\\MODELS\\fbcsp_mibif_cnn\\2a\\CURRENT\\tl_comp_csp_mi\\'
config_root= 'E:\\Diamond\\bci_iv\\MODELS\\fbcsp_mibif_cnn\\2a\\configs\\' #fixed
raw_data_root = 'E:\\Diamond\\bci_iv\\DATA\\2a\\extract_raw\\' #fixed


k_fold = 10


# initialize csp
m = 2# m is Nw in the paper "learning temporal information for brain-copmuter interface, Sakhavi et.al"
n_components = 2 * m  # pick some components
down_sample_step = 20 #Hilbert evelope
# select Ns pairs of csp filters
Ns = 4

CLASSES =[0,1]

C_OVR = [0,1,2,3]

balance_classes = 1


########################################################################################################################
                                            # DEFINE FILTER BANK
########################################################################################################################
#Filter Bank
FB = [[4., 8.], [8., 12.], [12., 16.], [16., 20.], [20., 24.], [24., 28.], [28., 32.], [32., 36.], [36., 40.]]
FB = np.array(FB)

#argumaents for Chebyl II filtering
# Nyquist frequency

# min. attenuation in stop band
gstop = 50
# max. attenuation in passband
gpass= 3 


EEG_PERIOD = [[2.5,6]]
FS = [250]
eeg_period = EEG_PERIOD[0]
fs = FS[0]
Nf = fs/2.0


to_center_matrix = 0
calc_kl_d = 0
calc_mi_discrim  =0

In [3]:
# read _lambda values
with open(config_root +'_lambda_config.csv', mode = 'r') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter = ',')
    for row in csv_reader:
        _lambda_list_str = row[1:]
csv_file.close()        

#create _lambda value in list
_lambda_list = []
for i in _lambda_list_str:
    _lambda_list.append(float(i))

In [4]:
_lambda_list

[0.0, 0.2, 0.4, 0.6, 0.8, 1.0]

#  CALCULATE COVARIENCE MATRICES FOR ALL SUBJECTS

In [5]:
to_save = 1

In [15]:
#t0  = datetime.datetime.now()
for subject in range(0,0):
    print (subject)
    filename_T = 'A0'+str(subject)+'T'
    filename_E = 'A0'+str(subject)+'E'
    #ilename_save = filename
    #rint (filename_save)
    
    file_root  = save_root + 'A0'+str(subject) + '\\4s\\'

    ########################################################################################################################
                                                        #LOAD DATA
    ########################################################################################################################
    #load raw eeg and labels
    EEG_extract_raw_T = pickle.load(open('E://Diamond//bci_iv//DATA//2a//extract_raw//' 
                                                            + filename_T + '.pickle', 'rb'),encoding='iso-8859-1')
    #Class labels
    LABELS_raw_T = pickle.load(open("E://Diamond//bci_iv//DATA//2a//extract_raw//" 
                                                        + filename_T + "_LABELS.pickle", 'rb'), encoding='iso-8859-1')


    EEG_extract_raw_E = pickle.load(open('E://Diamond//bci_iv//DATA//2a//extract_raw//' 
                                                            + filename_E + '.pickle', 'rb'),encoding='iso-8859-1')
    #Class labels
    LABELS_raw_E = pickle.load(open("E://Diamond//bci_iv//DATA//2a//extract_raw//" 
                                                        + filename_E + "_LABELS.pickle", 'rb'), encoding='iso-8859-1')

    ##concatenate training trials and evaluation trails into composite matrix
    EEG_extract_raw = np.concatenate((EEG_extract_raw_T,EEG_extract_raw_E),axis = 0)
    LABELS_raw = np.concatenate((LABELS_raw_T,LABELS_raw_E),axis = 0)


    LABELS = LABELS_raw - 1 #already in 1,2,3,4, the labels are. but we nned them to be likw 0,1,2,3

    #RUN THIS CODE ONLY ONCE, LABELS0 is the original labels, and LABELS will be changed in one-versus-rest strategy, later
    LABELS0 = LABELS.copy()

    ########################################################################################################################
                                        #APPLY FILTER BANK
    ########################################################################################################################
    #Store Filter bank filtered raw EEG data, in the shape of num_filter_bank X num_trials X num_chanl X num_samples
    #initiate empty matrix
    EEG_filt_FB_L = np.empty( [len(FB), 
                             np.shape(EEG_extract_raw)[0], np.shape(EEG_extract_raw)[1],np.shape(EEG_extract_raw)[2]] )

    for fb in range (0, len(FB)):
        passband = FB[fb]
        stopband = FB[fb] + np.array([-2., +2.])

        EEG_filt_FB_L[fb] = filter_signal(EEG_extract_raw, passband, stopband, Nf, gpass, gstop)

        
    EEG_filt_FB = EEG_filt_FB_L
    
    if center_matrix == 1:
        EEG_filt_FB = center_matrix(EEG_filt_FB)
    

    #trake only the MI 3.5 seconds
    EEG_filt_FB_go = EEG_filt_FB[:,:,:,int(eeg_period[0]*fs):int(eeg_period[1]*fs)]
    LABELS0_go = LABELS0.copy()

   
    ########################################################################################################################

    
    balance_classes = 1
        
    Covs = np.zeros((len(FB), len(C_OVR), 2, np.shape(EEG_filt_FB_go)[2], np.shape(EEG_filt_FB_go)[2]))
    for fb in range (0, len(FB)):
        for c_ovr in C_OVR:
            
            if balance_classes == 1:
                c_indx = make_ovr_sets_strat(c_ovr, C_OVR, LABELS0_go)
            else:
                c_indx = np.array(np.arange(0,len(LABELS0_go)))
            LABELS0_go_c_ovr = make_ovr_01_labels(LABELS0_go[c_indx].copy(), c_ovr)
            covs = covs_classes(_classes = [0,1], n_ch = np.shape(EEG_filt_FB_go)[2], X = EEG_filt_FB_go[fb][c_indx], y = LABELS0_go_c_ovr)
            Covs[fb][c_ovr] = covs
    if to_save == 1:
        pickle.dump(Covs, open(file_root + 'Covs.pickle', 'wb'))
    
    
    
    
    
    
    

2
3
4
5
6
7
8
9


In [6]:
save_root

'E:\\Diamond\\bci_iv\\MODELS\\fbcsp_mibif_cnn\\2a\\CURRENT\\tl_comp_csp_mi\\'

In [10]:
portion = 0.7
portion_train = str(int(portion*100)) #in the old versions of code portion_train is cv_train
cv_train_size = 0.7 #cv_train_size if the actually cv split



for k in range (1, 10):
    print (k)
    filename_T = 'A0'+str(k)+'T'
    file_root  = save_root + 'A0'+str(k) + '\\4s\\'
    
    k_file_root = file_root+'pt_'+ portion_train
    
    #set load_idx to 0 if we want to create new and overwirite old T_IND files
    load_idx = 0 # 0 to create and save new sets of train and test index, 1 to load already created indices
    COVS_AL_FD,  EEG_filt_FB_go, LABELS0_go, TRAIN_IDX, TEST_IDX, Train_idx = calc_k_covs_all_fold_current(filename_T, raw_data_root, 
                                                                                                            portion, k_file_root, 
                                                                                                            balance_classes, 
                                                                                                   FB,  gpass, gstop, fs, eeg_period, 
                                                                                                   k_fold, C_OVR, load_idx = load_idx, 
                                                                                                   cv_train_size = cv_train_size, 
                                                                                                 to_center_matrix = to_center_matrix)
     
    if to_save == 1 and load_idx == 0:
        pickle.dump(TRAIN_IDX , open(file_root +'pt_'+ portion_train + '\\TRAIN_IDX' + ".pickle", "wb" ) )

        pickle.dump(TEST_IDX , open(file_root +'pt_'+ portion_train + '\\TEST_IDX' + ".pickle", "wb" ) )

        pickle.dump(LABELS0_go , open(file_root +'pt_'+ portion_train + '\\LABELS0_go' + ".pickle", "wb" ) )
    
    MI_score_write = open(file_root+ '\\pt_' + portion_train + '\\MI_score.txt', "w")
    MI_score_write.write('')
    MI_score_write.close()
    MI_score_vals_write = open(file_root+ '\\pt_' + portion_train + '\\MI_score_vals.txt', "w")
    MI_score_vals_write.write('')
    MI_score_vals_write.close()
    
    ########################################################################################################################
    for fold in range (0, k_fold):
        Covs_k = COVS_AL_FD[fold]
        
        MI_score_write = open(file_root + '\\pt_' + portion_train + '\\MI_score.txt', "a")
        MI_score_write.write('fold,'+str(fold)+ '\n')
        MI_score_vals_write = open(file_root + '\\pt_' + portion_train + '\\MI_score_vals.txt', "a")
        MI_score_vals_write.write('fold,'+str(fold)+ '\n')
        
        for _lambda in _lambda_list:
            
            for c_ovr in C_OVR:
                W_B = []
                for i in range (0, len(FB)):
                    W_B.append([])
                    
                
                train_idx = Train_idx[fold][c_ovr]
                EEG_train = EEG_filt_FB_go[:, train_idx, :]    
                LABELS_train = LABELS0_go[train_idx]
                LABELS_train = make_ovr_01_labels(LABELS_train, c_ovr)
                
                V= [] 
                for tr in range (0, len(train_idx)):
                    V.append([])
                
                ########### how discriminative does subject l perform on subject k's data (Mutual information) ######### 
                for fb in range (0, len(FB)):
                    MI_SCORE = OrderedDict()

                    for l in range (1, 10):
                        if l != k:
                            #print (l)
                            V1 = []
                            for tr in range (0, len(train_idx)):
                                V1.append([]) 
                            
                            
                            Covs_l = pickle.load(open(save_root + 'A0'+str(l) + '\\4s\\' + 'Covs.pickle', 'rb'))

                            C_l = Covs_l[fb, c_ovr]

                            eigen_vectors_sorted, eigen_values = calc_sort_eigenvectors(C_l)
                            W_b_ = np.concatenate ( [eigen_vectors_sorted[:, 0:int(m)],  eigen_vectors_sorted[:, -int(m):]], axis = 1)
                            
                            for tr in range (0, len(train_idx)):
                                eeg = EEG_train[fb, tr, :]

                                #calculate features
                                v_bi = csp_features(W_b_, m, eeg)

                                #contruct feature matrix
                                V1[tr].append(v_bi)
                                
                            V1 = np.array(V1)
                            V1 = V1.reshape([np.shape(V1)[0], np.shape(V1)[1]*np.shape(V1)[2]]) 
                            F1 = np.transpose(V1)
                            ####CALCULATE MUTUAL INFORMATION FOR THE FEATURES
                            I_fj = calc_MI(FB[fb:fb+1], m, CLASSES, LABELS_train, F1)
                            I_fj_sorted = np.argsort(I_fj)[::-1]

                            mi_score = np.average(I_fj[I_fj_sorted[0:2*Ns]])

                            MI_SCORE[str(l)] = mi_score
                    
                    MI_SCORE_vals = np.array(list(MI_SCORE.values()))
                    range2 = [1,10]
                    
                    for key in MI_SCORE.keys():
                        MI_SCORE[key] = ((MI_SCORE[key] - min(MI_SCORE_vals)) / (max(MI_SCORE_vals) - min(MI_SCORE_vals))) * (range2[1] - range2[0]) + range2[0]
                    
                    
                    if _lambda == _lambda_list[0]:
                        count = 0
                        for i in MI_SCORE.keys():
                            MI_score_write.write ('sub,' + i + ',fold,' + str(fold) + ',c_ovr,' + str(c_ovr) + ',fb,' + str(fb) + ',mi_score,' + str(MI_SCORE[i]) + '\n')
                            MI_score_vals_write.write ('sub,' + i + ',fold,' + str(fold) + ',c_ovr,' + str(c_ovr) + ',fb,' + str(fb) + ',mi_score,' + str(MI_SCORE_vals[count])+ '\n')
                            count += 1
                    
                    #scale wieghts
                    weights = OrderedDict()
                    for key in MI_SCORE.keys():
                        weights[key] = MI_SCORE[key] / np.sum(np.array(list(MI_SCORE.values())))
                    
                    
                    sec_term = 0
                    for j in range (1,10):
                        if j != k:
                            Covs_j = pickle.load(open(save_root + 'A0'+str(j) + '\\4s\\' + 'Covs.pickle', 'rb'))
                            sec_term += weights[str(j)] * Covs_j[fb, c_ovr]

                            
                    ######################################################################################
                    ##### COMPOSITE CSP USING MUTUAL INFORMATION, transfer learning weighted by lambda
                    C_ck = (1-_lambda)*Covs_k[fb, c_ovr] + _lambda*sec_term 
                    
                    eigen_vectors_sorted, eigen_values = calc_sort_eigenvectors(C_ck)
                    W_b_ = np.concatenate ( [eigen_vectors_sorted[:, 0:int(m)],  eigen_vectors_sorted[:, -int(m):]], axis = 1)
                    W_B[fb].append(W_b_)
                    
                    for tr in range (0, len(train_idx)):
                        eeg = EEG_train[fb, tr, :]

                        #calculate features
                        v_bi = csp_features(W_b_, m, eeg)

                        #contruct feature matrix
                        V[tr].append(v_bi)
                        
                V = np.array(V)
                V = V.reshape([np.shape(V)[0], np.shape(V)[1]*np.shape(V)[2]]) 
                W_B = np.array(W_B).squeeze()

                #follow MIBIF Algorithm in bci_iv_fbcsp paper
                F = np.transpose(V)  

                ####CALCULATE MUTUAL INFORMATION FOR THE FEATURES
                I_fj = calc_MI(FB, m, CLASSES, LABELS_train, F)

                ####SORT FEATURES ACCORDING TO MUTAUL INFORMATION
                # I_fj_sorted is the indices of features ranked by decending mutal information, in shape (num_features, );
                #where num_features = 2* m * num_FB
                I_fj_sorted = np.argsort(I_fj)[::-1]


                #the filter bank indices from which the features come from, in shape (num_features, )
                FB_IND = np.floor(I_fj_sorted/(2*m)).astype(int)
                #the filter channel (csp.eigen_vectors_sorted column) indiceis from which the features come from, in shape (num_features, )
                FILTER_IND = (I_fj_sorted) - FB_IND * (2*m)

                #combine the filterbank indicies and the CSP filter indicies, 
                #first column tells which filter bank the feature came from, 
                #second column tells which csp filter channel the feature came from
                FB_FILTER_IND = np.array(list(zip(FB_IND, FILTER_IND)))
                
                if to_save == 1:
                    pickle.dump( W_B, open(file_root + 'pt_' + portion_train
                                                    + '\\W_B_fold_' + str(fold) + 
                                           '_c_ovr_' + str(c_ovr) + '_lambda_' + str(_lambda) +
                                           ".pickle", "wb" ) )

                    pickle.dump( FB_FILTER_IND, open(file_root + 'pt_' + portion_train + '\\FB_FILTER_IND_fold_' + str(fold) + 
                                           '_c_ovr_' + str(c_ovr) + '_lambda_' + str(_lambda) +
                                           ".pickle", "wb" ) )
                    
                
    ########################################################################################################################

    for fold in range (0, k_fold):
        #print ('fold', fold)
        for _lambda in _lambda_list:
            for c_ovr in C_OVR:
                
                #load in csp filters and mutual informtaion ranked indicies
                W_B = pickle.load(open( file_root + 'pt_' + portion_train
                                                    + '\\W_B_fold_' + str(fold) + 
                                           '_c_ovr_' + str(c_ovr) + '_lambda_' + str(_lambda) + 
                                           ".pickle", 'rb'))


                FB_FILTER_IND = pickle.load(open( file_root + 'pt_' + portion_train
                                                    + '\\FB_FILTER_IND_fold_' + str(fold) + 
                                           '_c_ovr_' + str(c_ovr) + '_lambda_' + str(_lambda) + 
                                           ".pickle", 'rb'))
                
                
                #find the selected csp filters indicies
                FB_FILTER_IND_slt = find_selected_csp_filters(Ns, m, FB_FILTER_IND)

                #construct selected csp filters, W_B_slt has shape (2*Ns, num_chls), (8,22) for example
                W_B_slt = W_B[FB_FILTER_IND_slt[:,0], :, FB_FILTER_IND_slt[:,1]]



                #load in training eeg signals and testing EEG
                train_idx = TRAIN_IDX[fold]
                test_idx = TEST_IDX[fold]
                EEG_train_FB = EEG_filt_FB_go[:,train_idx,:]
                EEG_test_FB = EEG_filt_FB_go[:,test_idx,:]
                #pick only the eeg signals filtered by selected filter banks
                EEG_train_FB_slt = EEG_train_FB[FB_FILTER_IND_slt[:,0], :]
                EEG_test_FB_slt = EEG_test_FB[FB_FILTER_IND_slt[:,0], :]

                #transform into z space, then take the hilbert envelope of the transformed signal
                Z_env_train = calc_z_features(W_B_slt, EEG_train_FB_slt, Ns, down_sample_step)
                Z_env_test = calc_z_features(W_B_slt, EEG_test_FB_slt, Ns, down_sample_step)

                #concatenate all classes
                if c_ovr == C_OVR[0]:
                    Z_all_classes_train = Z_env_train
                    Z_all_classes_test = Z_env_test
                else:
                    Z_all_classes_train = np.concatenate((Z_all_classes_train, Z_env_train), axis = 0)
                    Z_all_classes_test = np.concatenate((Z_all_classes_test, Z_env_test), axis = 0)
                    
            #trasnpose into shape (num_trials, num_features, sample points)
            Z_all_classes_train = np.transpose(Z_all_classes_train, [1,0,2])
            Z_all_classes_test = np.transpose(Z_all_classes_test, [1,0,2])

            #save the Z space hilbert envelop signals
            if to_save == 1:
                pickle.dump( Z_all_classes_train, open(file_root +'pt_' + portion_train
                                                    + '\\Z_all_classes_train_fold_' + str(fold) + '_lambda_' + str(_lambda) + ".pickle", "wb" ) )

                pickle.dump( Z_all_classes_test, open(file_root +'pt_' + portion_train
                                                    + '\\Z_all_classes_test_fold_' + str(fold) + '_lambda_' + str(_lambda) + ".pickle", "wb" ) )



    if to_save == 1:
        MI_score_write.close()
        MI_score_vals_write.close()


1
2
3
4
5
6
7
8
9


In [12]:
MI_SCORE_vals

array([-13.18607994, -14.71548466, -14.51332725, -14.66271145,
       -13.89988831, -14.20405894, -15.12390236, -13.65705057])

In [11]:
print (portion_train)

70


In [48]:
MI_SCORE_vals

array([-77.3416249 , -76.1576916 , -79.33752962, -75.94727417,
       -78.76682398, -74.52494616, -78.31712559, -78.77128242])

In [49]:
count = 0
for i in MI_SCORE.keys():
    MI_score.write ('sub,' + i + ',fold,' + str(fold) + ',c_ovr,' + str(c_ovr) + ',fb,' + str(fb) + ',mi_score,' + str(MI_SCORE[i]) + '\n')
    MI_score_vals.write ('sub,' + i + ',fold,' + str(fold) + ',c_ovr,' + str(c_ovr) + ',fb,' + str(fb) + ',mi_score,' + str(MI_SCORE_vals[count])+ '\n')
    count += 1

sub,1,fold,9,c_ovr,3,fb,8,mi_score,4.732536297144877
sub,1,fold,9,c_ovr,3,fb,8,mi_score,-77.34162489984658
sub,2,fold,9,c_ovr,3,fb,8,mi_score,6.946606932573529
sub,2,fold,9,c_ovr,3,fb,8,mi_score,-76.1576915977171
sub,3,fold,9,c_ovr,3,fb,8,mi_score,1.0
sub,3,fold,9,c_ovr,3,fb,8,mi_score,-79.33752961620235
sub,4,fold,9,c_ovr,3,fb,8,mi_score,7.340108024991786
sub,4,fold,9,c_ovr,3,fb,8,mi_score,-75.94727417006891
sub,5,fold,9,c_ovr,3,fb,8,mi_score,2.067275149382838
sub,5,fold,9,c_ovr,3,fb,8,mi_score,-78.7668239794127
sub,6,fold,9,c_ovr,3,fb,8,mi_score,10.0
sub,6,fold,9,c_ovr,3,fb,8,mi_score,-74.52494615627928
sub,7,fold,9,c_ovr,3,fb,8,mi_score,2.9082549572545835
sub,7,fold,9,c_ovr,3,fb,8,mi_score,-78.31712558902461
sub,8,fold,9,c_ovr,3,fb,8,mi_score,2.0589374355800403
sub,8,fold,9,c_ovr,3,fb,8,mi_score,-78.77128241758392


In [35]:
6.946606932573529/4.732536297144877

1.4678401804893484

In [47]:
umm = MI_SCORE_vals
umm/np.sum(umm)

array([0.12491293, 0.12300078, 0.12813647, 0.12266094, 0.12721474,
       0.12036377, 0.12648844, 0.12722194])

In [46]:
MI_SCORE_vals

array([-77.3416249 , -76.1576916 , -79.33752962, -75.94727417,
       -78.76682398, -74.52494616, -78.31712559, -78.77128242])

In [34]:
-77.3416249/-76.1576916

1.0155458138912392

In [19]:
save_root + 'A0'+str(0) + '\\4s\\'

'E:\\Diamond\\bci_iv\\MODELS\\fbcsp_mibif_cnn\\2a\\CURRENT\\tl_comp_csp_mi\\A00\\4s\\'

In [8]:
print('goog')

goog
